Кобзарь О.С. Хабибуллин Р.А.
# Обработчик данных 

В данной тетрадке собраны все средства для работы с данными (пре- и пост- процессор; генерация входных данных для модели, анализ рассчитанных значений, построение графиков)

Большое количество кода вынесено в модуле - здесь в основном управление

`well_name` - номер исследуемой скважины, например `1`

Исходными данные лежат в папке  `data` в папках, соответствующим названиям скважин. Это данные
1. Со станции управления (`data/well_name/well_name.csv`). Большое количество записей, которые нужно преобразовать в "шахмоткоподобный" вид для дальнейшей работы. Данные со СУ могут быть как высокочастотными (токи, напряжения), так и низкочастотными. 
2. С шахматки `data/well_name/Скв. well_name (01.07.2018-31.03.2019).xls`. Низкочастотные данные. 
3. C техрежима `data/tr/Техрежим, , февраль 2019.xls`. Какой насос спущен, ПЭД, на какую глубину. Эти данные постоянны на всем периоде расчета. В одном файлике один месяц и данные для всех скважин.



## Общий workflow
Здесь кратко, подробное описание будет над ячейками.
1. Обработка исходных данных со СУ и приведение их в удобный формат. (init_edit)
2. Считавание подготовленных ранее данных со СУ, шахматки, а затем генерации входных данных для модели, конкретно, адаптации модели скважины за выбранный период времени. (adaptation_input)
3. Адаптация. Теперь можно открыть файл processor.py и в нем, выставив настройки и определив входные данные, запустить расчет скважину. После адаптации данные появится данные с калибровочными коэффициентами. (adaptation)
4. Считывание рассчитанных значений адаптации, данных со СУ и шахматки. Генерации файлов для восстановления дебитов (restore_input)
5. Восстановление. Работа в processor.py, где необходимо изменить метод расчета и указать на новые входные данные. (restore) 
6. Заключительный анализ: сведение результатов адаптации и восстановления; расчет различных метрик для оценки метода (тоже в папке restore)

По идее, если данные уже сгенерированы, каждый пункт может повторяться многократно

Импорт модулей. Некоторые функции и методы вынесены в отдельные `.py`
* `preprocessor` - для обработки данных в и из модели
* `processor` - для непосредственного запуска расчета адаптации или восстановления
* `postprocessor` - для завершающего анализа сгенерированных данных. Определение метрик успешности модели, паттернов, событий, зависимостей.
* `plotly_workflow` - для быстрого построения шаблонизированных графиков

In [ ]:
import os
import preprocessor as prep
import postprocessor as postp
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from sklearn import metrics

Общие настройки и флаги для работы. Указание номера скважины и название файла шахматки. Флаги для запуска тех или иных ячеек

In [ ]:
well_name = '569'
chess_file_name = 'Скв. ' + well_name + ' (01.07.2018-31.03.2019).xls'

# TODO убрать флаги
read_initial_data = True
plot_initial_data = True
create_input_data = True
multiprocessing_on = True #TODO может быть убрать флаги, как лучше?
created_input_data_type = 0 # 0 - adaptation input, 1 - restore input with calibation coefficients

Определение путей к данным. Создание метки времени к генерируемым папкам. (чтоб данные не перезаписывались)

In [ ]:
current_path = os.getcwd()
time_mark = datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
save_dir_name = 'init_edit'
path_to_save = path_to_work_dir + save_dir_name + '\\'
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)
cs_data_filename = path_to_save + well_name + "_first_edit.csv"

Задание интервалов, на которых будет вестить расчет. Их может быть несколько. Рекомендуется поставить малый период времени, выявить ошибки, а затем запускать на месяцы и годы

In [ ]:
left_boundary = [datetime.datetime(2019,1,24)]#, datetime.datetime(2018,11,29)]
right_boundary = [datetime.datetime(2019,2,25)]#, datetime.datetime(2019,2,28)]

# 1. Обработка исходных данных
Чтение исходных данных `well_name.csv` со СУ и преобразование их в удобный формат. Процесс небыстрый из-за несовершенства алгоритма форматирования и количества данных (около 1-1,5 млн. записей)

In [ ]:
%%time
if read_initial_data:
    try:
        os.mkdir(path_to_work_dir + save_dir_name)
    except:
        pass
    data_file_path = path_to_work_dir + well_name + ".csv"
    well_data = pd.read_csv(data_file_path,sep=';', header=None)
    well_data = prep.initial_editing(well_data, well_name)
    well_data = prep.create_edited_df(well_data)


Сохранение обработанных данных со СУ в папку `init_edit`

In [ ]:
%%time
well_data.to_csv(path_to_save + well_name + "_first_edit.csv")

Построение исходных данных со станции управления через `plotly`. Сохранение в ту же папку. Процесс долгий опять же из-за большого количества точек. Ориентировочное время - 5-6 минут - можно за чаем сходить

In [ ]:
%%time
plot_file_path = path_to_save + well_name + '_init_sp.html'
well_data_traces = pw.create_traces_list_for_all_columms(well_data, 'lines+markers', use_gl = True)
pw.plot_subplots(well_data_traces, plot_file_path, True)

На этом этапе в папке init_edit окажутся
1. well_name_init_edit.csv - данные с шахматки, которые будут подгружаться для адаптации и восстановления
2. well_name_init_sp.html - график для исходных данных

Перейдем к следующему этапу

# 2. Генерация входных данных для адаптации

Для данного этапа уже нужные обработанные данные со станции управления.

Чтение исходных данных со СУ (в уже подготовленных). Обработка этих данных - ресемпл, пометка названий столбцов с помощью `(СУ)`. Т.к. данные высокочастотные, осредним их по 3 часам - примерному времени замера дебита скважины на АГЗУ. Обработка будет отличаться в зависимости от того, какие данные вы генерируете - для адаптации выбрасываются интервалы без замеров дебитов, для восстановления - нет

In [ ]:
time_to_resamle = '3h'
created_input_data_type = 0 

In [ ]:
%%time
edited_data_cs = prep.load_and_edit_cs_data(cs_data_filename,
                                       time_to_resamle, 
                                       created_input_data_type)

Чтение данных с шахматки. Осреднение и протяжка параметров. Пометка названий столбцов маркером `(Ш)`

In [ ]:
%%time
chess_data = prep.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                     time_to_resamle)

Генерация входных данных для адаптации. Сведение данных с шахматки и со СУ. Создание папки с названием `adapt_input_` и временной пометкой и помещение в нее:
* сгенерированных входных данных для модели `well_name_adapt_input.csv`
* всех входных данных на графике `well_name_adapt_input.html`

In [ ]:
input_data_dir_name = 'adapt_input_' + time_mark
path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
if create_input_data:
    try:
        os.mkdir(path_to_work_dir + input_data_dir_name)
    except:
        pass
created_input_data = edited_data_cs.join(chess_data, how = 'outer')
created_input_data = created_input_data.dropna(subset = ['Объемный дебит жидкости (СУ)'])
created_input_data = prep.cut_df(created_input_data, left_boundary, right_boundary)
plot_file_path = path_to_input_data + well_name + '_adapt_input.html'
input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', use_gl = True)
pw.plot_subplots(input_data_traces, plot_file_path, True)

created_input_data.to_csv(path_to_input_data + well_name + '_adapt_input.csv')

Построение кривых в отчете только для тех данных, которые будут использоваться в модели в `well_name_adapt_input_report.html` в той же папке в стандартном виде. Можно настроить, какие данные выводить.

In [ ]:
created_input_data['Линейное давление (СУ)'] =  created_input_data['Линейное давление (СУ)'] * 10
qliq = {'Объемный дебит жидкости (СУ)':['Объемный дебит жидкости (СУ)']}
pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)']}
pressure_wh = {'Рлин ТМ (Ш)': ['Рлин ТМ (Ш)', 'Линейное давление (СУ)']}
pressure_bf = {'Рбуф (Ш)': ['Рбуф (Ш)']}
temp_intake = {'Температура на приеме насоса (пласт. жидкость) (СУ)': ['Температура на приеме насоса (пласт. жидкость) (СУ)']}
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)']}
choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}
gor = {'ГФ (СУ)':['ГФ (СУ)']}
wc = {'Процент обводненности (СУ)':['Процент обводненности (СУ)']}
all_banches = [qliq, gor,wc, pressure_intake, pressure_wh, pressure_bf,
               temp_intake, frequencies, choke,power,voltage,cos]

pw.create_report_html(created_input_data, all_banches, path_to_input_data + well_name + '_adapt_input_report.html')

# 3. Адаптация

Нужно запустить модуль `processor.py` и начать в нем адаптацию, используя `well_name_adapt_input.csv`

# 4. Генерация данных для восстановления дебитов

## 4.1 Загрузка и анализ адаптации
После успешной адаптации нужно проанализировать ее корректность, построив графики. Для этого нужно указать директории с 
* результатами адапатации - `adaptation_time_mark`  
* входными данными для адаптации - `adaptation_input_time_mark`

In [ ]:
dir_name_with_input_data = 'adapt_input_2019_11_14_17_07_45' + '\\'
input_data_file_name = well_name + '_adapt_input'
dir_name_with_calculated_data = 'adaptation_2019_11_14_17_31_40' + '\\'
calculated_data_file_name = well_name + '_adapt_1'

Если расчет проводился в многопоточном варианте, то загрузка будет отличаться. В папке с расчетами `adaptation` в сабдиректории `multiprocessing` будут результаты по частям в разных файликах, поэтому их нужно собрать в один файлик и поместить на уровень выше в `adaptation`. Если расчет был без использования многопоточности, этот шаг можно пропустить.

In [ ]:
if multiprocessing_on == True:
    filenames_list = []
    for (dirpath, dirnames, filenames) in os.walk(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\'):
        filenames_list.extend(filenames)
        break
    print(filenames_list)
    for i,j in enumerate(filenames_list):
        if i == 0:
            first_result_data = pd.read_csv(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
        else:
            another_result_data = pd.read_csv(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
            first_result_data = first_result_data.append(another_result_data, sort = True)
            del first_result_data['d']
            first_result_data = first_result_data.dropna(subset = ['ESP.ESPpump.EffiencyESP_d'])
    first_result_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + calculated_data_file_name + '.csv')

Загрузка данных адаптации после расчета - в папке `adaptation_time_mark`  файл `well_name_adapt_1.csv`

In [ ]:
calculated_data = prep.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv')

Чтение входных данных для адаптации из папки `adaptation_input` и объединение данных адатации и входных данных для нее.
Сохранение всех данных в папке `adaptation_time_mark` с названием `well_name_calc_and_input.csv`

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv')
input_data.index = input_data['Время']
del input_data['Время']
all_data = input_data.join(calculated_data, how = 'outer')

all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

Построение графиков для данных адаптации в папке  `adaptation_time_mark`

Все данные (входные и результаты) на одном графике `well_name_calc_and_input.html`

In [ ]:
adapt_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pw.plot_subplots(adapt_data_traces, plot_file_path, True)

Создание отчета - набор нужных графиков для быстрой оценки результатов адаптации

In [ ]:
qliq = {'Объемный дебит жидкости':['Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)']}
calibrs = {'Калибровки по напору и мощности':
       ['К. калибровки по напору - множитель (Модель)',
           'К. калибровки по мощности - множитель (Модель)']}
metrics = {'Метрики расчета':
       ['Значение функции ошибки (Модель)']}
gor = {'ГФ (Модель)':['ГФ (Модель)']}
wc = {'Обводненность, %':['Обв, % (Модель)']}
pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)', 'P прием ЭЦН, атм (Модель)']}
pressure_wh = {'Рлин': ['Рлин ТМ (Ш)', 'P лин., атм (Модель)']}
pressure_bf = {'Рбуф': ['Рбуф (Ш)', 'P буф., атм (Модель)']}
temp_intake = {'Температура на приеме насоса': ['Температура на приеме насоса (пласт. жидкость) (СУ)',
                                                                      'T прием ЭЦН, C (Модель)']}
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)', 'F тока, ГЦ (Модель)']}
choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
true_power = {'Мощность, передаваемая СУ (Модель)':['Мощность, передаваемая СУ (Модель)']}
voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}

efficiency = {'КПД ЭЦН, д.ед.':
          ['КПД ЭЦН, д.ед. (Модель)']}
dif_pressure = {'Перепад давления в ЭЦН, атм':
            ['Перепад давления в ЭЦН, атм (Модель)']}
all_banches = [qliq,calibrs,metrics, gor,wc, pressure_intake, pressure_wh, pressure_bf,
               temp_intake, frequencies, choke,power,true_power, voltage,cos, efficiency, dif_pressure]

pw.create_report_html(all_data, all_banches, path_to_work_dir + dir_name_with_calculated_data + 
                      well_name + '_adapt_report.html')

# 4.2 Генерации данных для восстановления дебитов
Работа с калибровками для восстановления дебитов. Для преобразования их выделим в отдельный DataFrame. Будем интерполировать, выкалавать точки, либо еще что-нибудь

In [ ]:
calibr_data = calculated_data[['К. калибровки по напору - множитель (Модель)',
                               'К. калибровки по мощности - множитель (Модель)']]
calibr_data = prep.mark_df_columns(calibr_data, 'Подготовленные')
#calibr_data = calibr_data.resample('3h').mean()
#calibr_data = calibr_data.interpolate()

In [ ]:
result = prep.make_gaps_and_interpolate(calibr_data)
calibr_data = result.copy()
all_data = all_data.join(calibr_data, how = 'inner')
created_input_data = all_data.copy()

Прочитаем данные со СУ и шахматки и обработаем их, для генерации входных данных модели

In [ ]:
time_to_resamle = '3h'
created_input_data_type = 1 

In [ ]:
edited_data_cs = prep.load_and_edit_cs_data(cs_data_filename,
                                       time_to_resamle, 
                                       created_input_data_type)
chess_data = prep.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                     time_to_resamle)

Генерация входных данных для модели для восстановления дебитов в папку `restore_input_time_mark`
* `well_name_restore_input.csv` - входные данных с калибровками (увеличили частоту дискретизации или выкололи точки)
* `well_name_restore_input.html` - все входные данные на одном графике

In [ ]:
input_data_dir_name = 'restore_input_' + time_mark
path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
if create_input_data:
    try:
        os.mkdir(path_to_work_dir + input_data_dir_name)
    except:
        pass
created_input_data = edited_data_cs.join(chess_data, how = 'outer')
created_input_data = created_input_data.join(calibr_data, how ='inner')
created_input_data = prep.cut_df(created_input_data, left_boundary, right_boundary)
plot_file_path = path_to_input_data + well_name + '_restore_input.html'
input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', use_gl = True)
pw.plot_subplots(input_data_traces, plot_file_path, True)
created_input_data.to_csv(path_to_input_data + well_name + '_restore_input.csv')

Построение графиков в отчетной форме 

In [ ]:

qliq = {'Объемный дебит жидкости':['Объемный дебит жидкости (СУ)']}
calibrs = {'Калибровки по напору и мощности':
       ['К. калибровки по напору - множитель (Модель) (Подготовленные)',
           'К. калибровки по мощности - множитель (Модель) (Подготовленные)']}
gor = {'ГФ (СУ)':['ГФ (СУ)']}
wc = {'Процент обводненности (СУ)':['Процент обводненности (СУ)']}
pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)']}
pressure_wh = {'Рлин': ['Рлин ТМ (Ш)']}
pressure_bf = {'Рбуф': ['Рбуф (Ш)']}
temp_intake = {'Температура на приеме насоса': ['Температура на приеме насоса (пласт. жидкость) (СУ)']}
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)']}
choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}

all_banches = [qliq,calibrs, gor,wc, pressure_intake, pressure_wh, pressure_bf,
               temp_intake, frequencies, choke,power, voltage,cos]

pw.create_report_html(created_input_data, all_banches, path_to_input_data  + 
                      well_name + '_restore_input_report.html')

# 5. Восстановление дебитов.
Нужно, используя сгенерированные данные `well_name_restore_input` восстановить дебиты с помощью `postprocessor.py`
Для этого активировать флаги
* vfm_calc_option
* restore_q_liq_only

и определить папку с входными данными `dir_name_with_input_data`

# 6. Работа с данными после восстановления дебитов

# 6.1 Первичная обработка результатов восстановления
Обозначим директории с результатами восстановления дебитов и входными данными

In [ ]:
dir_name_with_input_data = 'restore_input_2019_11_14_17_38_59' + '\\'
input_data_file_name = well_name +'_restore_input'
dir_name_with_calculated_data = 'restore_2019_11_14_17_40_37' + '\\'
calculated_data_file_name = well_name +'_restore_1'

Объединение данных при запуске расчета в многопотоке. Если расчет проводился без многопоточности, этот шаг можно пропустить.

In [ ]:
if multiprocessing_on == True:
    filenames_list = []
    for (dirpath, dirnames, filenames) in os.walk(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\'):
        filenames_list.extend(filenames)
        break
    print(filenames_list)
    for i,j in enumerate(filenames_list):
        if i == 0:
            first_result_data = pd.read_csv(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
        else:
            another_result_data = pd.read_csv(path_to_work_dir + dir_name_with_calculated_data + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
            first_result_data = first_result_data.append(another_result_data, sort = True)
            del first_result_data['d']
            first_result_data = first_result_data.dropna(subset = ['ESP.ESPpump.EffiencyESP_d'])
    first_result_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + calculated_data_file_name + '.csv')

Загрузка данных после восстановления дебитов

In [ ]:
calculated_data = prep.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv')

Объединение входных и выходных данным модели. Сохранение результатов в `well_name_calc_and_input.csv`

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv', parse_dates = True, index_col = 'Время')
all_data = input_data.join(calculated_data, how = 'outer')
all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

Построение графиков восстановления дебитов

In [ ]:
input_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pw.plot_subplots(input_data_traces, plot_file_path, True)

# 6.1 Сведение данных адаптации и восстановления
Использование файлов типа `calc_and_input` в папках с адаптацией и восстновлением для формирования общего отчета

In [ ]:
path_to_adapt_dir = 'adaptation_2019_11_14_17_31_40' + '\\'
path_to_restore_dir = 'restore_2019_11_14_17_40_37' + '\\'

Загрузка и слияние данных адаптации и восстановления

In [ ]:
adapt_data_with_input = pd.read_csv(path_to_work_dir + path_to_adapt_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
adapt_data_with_input = prep.mark_df_columns(adapt_data_with_input, 'ADAPT')
restore_data_with_input = pd.read_csv(path_to_work_dir + path_to_restore_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
restore_data_with_input = prep.mark_df_columns(restore_data_with_input, 'RESTORE')
overall_data = adapt_data_with_input.join(restore_data_with_input, how = 'outer')

Получение дебитов методом линейной интерполяции

In [ ]:
q_liq = overall_data[['Объемный дебит жидкости (СУ) (ADAPT)', 'Активная мощность (СУ) (ADAPT)']]
result = prep.make_gaps_and_interpolate(q_liq)
result = prep.mark_df_columns(result, 'INTERP')
overall_data = overall_data.join(result)

Заключительная обработка данных

In [ ]:
overall_data['Активная мощность (СУ) (ADAPT)'] = overall_data['Активная мощность (СУ) (ADAPT)'] * 1000
overall_data['Активная мощность (СУ) (ADAPT) (INTERP)'] = overall_data['Активная мощность (СУ) (ADAPT) (INTERP)'] * 1000
overall_data['Загрузка двигателя (СУ) (ADAPT)'] = overall_data['Загрузка двигателя (СУ) (ADAPT)'] / 100
overall_data['Относительная ошибка расчетов (Q ж), %'] = postp.relative_error_perc(overall_data['Объемный дебит жидкости (СУ) (ADAPT)'],
                                                                      overall_data['Q ж, м3/сут (Модель) (RESTORE)']).abs()
overall_data['Относительная ошибка расчетов (N акт), %'] = postp.relative_error_perc(overall_data['Активная мощность (СУ) (ADAPT)'],
                                                                      overall_data['Мощность, передаваемая СУ (Модель) (RESTORE)']).abs()
overall_data['Относительная ошибка расчетов (Q ж) (INTERP), %'] = postp.relative_error_perc(overall_data['Объемный дебит жидкости (СУ) (ADAPT)'],
                                                                      overall_data['Объемный дебит жидкости (СУ) (ADAPT) (INTERP)']).abs()
overall_data['Относительная ошибка расчетов (N акт) (INTERP), %'] = postp.relative_error_perc(overall_data['Активная мощность (СУ) (ADAPT) (INTERP)'],
                                                                      overall_data['Мощность, передаваемая СУ (Модель) (RESTORE)']).abs()

Подготовка вывода в отчет

In [ ]:
liquid_rates = {'Дебиты': 
                ['Qж ТМ (Ш) (ADAPT)','Объемный дебит жидкости (СУ) (ADAPT)', 'Q ж, м3/сут (Модель) (ADAPT)',
                  'Объемный дебит жидкости (СУ) (ADAPT) (INTERP)', 'Q ж, м3/сут (Модель) (RESTORE)'] }

calibrs = {'Калибровки по напору и мощности':
           ['К. калибровки по напору - множитель (Модель) (ADAPT)',
               'К. калибровки по мощности - множитель (Модель) (ADAPT)', 
                  'К. калибровки по напору - множитель (Модель) (RESTORE)',
               'К. калибровки по мощности - множитель (Модель) (RESTORE)']}

gor_wc = {'ГФ, м3/м3 и Обводненность, %':
          ['ГФ (Модель) (ADAPT)', 'Обв, % (Модель) (ADAPT)',
                  'ГФ (Модель) (RESTORE)', 'Обв, % (Модель) (RESTORE)']}

temperatures = {'Температура, С':
                ['T устья, С (Модель) (ADAPT)', 'T устья, С (Модель) (RESTORE)', 
                'T прием ЭЦН, C (Модель) (ADAPT)', 'T прием ЭЦН, C (Модель) (RESTORE)']}

pressures_up = {'Давления (Устьевое и буферное), атм':
                ['Рбуф (Ш) (ADAPT)', 'P буф., атм (Модель) (ADAPT)', 'P буф., атм (Модель) (RESTORE)',
             'Рлин ТМ (Ш) (ADAPT)', 'P лин., атм (Модель) (ADAPT)', 'P лин., атм (Модель) (RESTORE)']}

pressures_down = {'Давления (На приеме), атм':
                  ['P прием ЭЦН, атм (Модель) (ADAPT)', 'P прием ЭЦН, атм (Модель) (RESTORE)']}
                 
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш) (ADAPT)', 'Выходная частота ПЧ (СУ) (ADAPT)',
               'F тока, ГЦ (Модель) (ADAPT)', 'F тока, ГЦ (Модель) (RESTORE)']}

beam_sizes = {'Размер штуцера, мм':
              ['Dшт (Ш) (ADAPT)', 'Dшт (Ш) (RESTORE)']}

powers = {'Активная мощность, Вт':
          ['Активная мощность (СУ) (ADAPT)', 'Мощность, передаваемая СУ (Модель) (ADAPT)',
           'Активная мощность (СУ) (ADAPT) (INTERP)', 'Мощность, передаваемая СУ (Модель) (RESTORE)']}
                       
currents= {'Токи, А':
           ['Ток фазы А (СУ) (ADAPT)', 'I, А (Модель) (ADAPT)', 'I, А (Модель) (RESTORE)']}
               
loads = {'Загрузка, д.ед.':
         ['Загрузка двигателя (Модель) (ADAPT)', 'Загрузка двигателя (Модель) (RESTORE)']}

efficiency = {'КПД ЭЦН, д.ед.':
              ['КПД ЭЦН, д.ед. (Модель) (ADAPT)', 'КПД ЭЦН, д.ед. (Модель) (RESTORE)']}

metrics = {'Метрики расчета':
           ['Значение функции ошибки (Модель) (ADAPT)',
                  'Значение функции ошибки (Модель) (RESTORE)',
          'Относительная ошибка расчетов (Q ж), %',
            'Относительная ошибка расчетов (Q ж) (INTERP), %',
          'Относительная ошибка расчетов (N акт), %',
           'Относительная ошибка расчетов (N акт) (INTERP), %']}

dif_pressure = {'Перепад давления в ЭЦН, атм':
                ['Перепад давления в ЭЦН, атм (Модель) (ADAPT)', 'Перепад давления в ЭЦН, атм (Модель) (RESTORE)']}

all_banches = [liquid_rates,calibrs, metrics, gor_wc, temperatures,pressures_up, 
               pressures_down,frequencies, beam_sizes,powers, currents,loads,efficiency, dif_pressure]

Создание сводного отчета `well_name_adapt_and_restore_report.html`

In [ ]:
plot_file_path = path_to_work_dir + path_to_restore_dir + well_name + '_adapt_and_restore_report' +  '.html'
pw.create_report_html(overall_data, all_banches, plot_file_path)

Расчет различных метрик для модели и сохранение их в текстовый файл `well_name_adapt_and_restore_metrics_report.txt`

In [ ]:
overall_data_with_calibr_gaps = overall_data[overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)'] !=
                                            overall_data['К. калибровки по напору - множитель (Модель) (RESTORE)']]
overall_data_with_calibr_gaps = overall_data_with_calibr_gaps.dropna(subset = ['Q ж, м3/сут (Модель) (RESTORE)'])
calibr_calc_metrics = postp.calc_mertics(overall_data_with_calibr_gaps['Q ж, м3/сут (Модель) (ADAPT)'],
            overall_data_with_calibr_gaps['Q ж, м3/сут (Модель) (RESTORE)'], 'Метрики восстановления Qж с помощью калибровок')
calibr_calc_metrics += 'Средняя относительная ошибка Q ж, %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (Q ж), %'].abs().mean()) + '\n'
calibr_calc_metrics += 'Средняя относительная ошибка N акт, %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (N акт), %'].abs().mean()) + '\n'

interp_calc_metrics = postp.calc_mertics(overall_data_with_calibr_gaps['Объемный дебит жидкости (СУ) (ADAPT)'],
            overall_data_with_calibr_gaps['Объемный дебит жидкости (СУ) (ADAPT) (INTERP)'], 'Метрики восстановления Qж с помощью интерполяции')
interp_calc_metrics +='Средняя относительная ошибка Q ж (INTERP), %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (Q ж) (INTERP), %'].abs().mean()) + '\n' 
interp_calc_metrics +='Средняя относительная ошибка N акт (INTERP), %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (N акт) (INTERP), %'].abs().mean()) + '\n'

In [ ]:
metrics_text_file_path = path_to_work_dir + path_to_restore_dir + well_name + '_adapt_and_restore_metrics_report' +  '.txt'
text_file = open(metrics_text_file_path, "w")
text_file.write(calibr_calc_metrics + '\n' + interp_calc_metrics)
text_file.close()

# Конец
Основной цикл тетрадки закачивается. Дальше построение полезных графиков, их нужно распихать по модулям и в общий workflow. Тут шаблоны для построения корреляционной и тепловых карт в разном виде, построение напорной характеристики ЭНЦ, обезразмеревание величин

Построение напорной характеристики ЭЦН

In [ ]:
q_esp_nom_m3day = 320
head_esp_nom_m = 1200

import sys
import pandas as pd
import os
sys.path.append('C:\\Users\\olegk\\Documents\\')
import unifloc_vba.description_generated.python_api as python_api
path_to_addin = os.getcwd()
path_to_addin = path_to_addin.replace('unifloc\\sandbox\\uTools', 'unifloc_vba\\UniflocVBA_7.xlam')
UniflocVBA = python_api.API(path_to_addin)
h_m = []
efficency_perc = []
power_wt = []
q_m3day = list(range(1, int(q_esp_nom_m3day * 1.8),10))
pump_id = UniflocVBA.calc_ESP_id_by_rate(q_esp_nom_m3day)
num_stages = int(head_esp_nom_m / UniflocVBA.calc_ESP_head_m(q_esp_nom_m3day, pump_id = pump_id))
for i in q_m3day:
    h_m.append(UniflocVBA.calc_ESP_head_m(i,num_stages = num_stages,  pump_id = pump_id) / 100)
    power_wt.append(UniflocVBA.calc_ESP_power_W(i, num_stages = num_stages, pump_id = pump_id) / 1000)
    efficency_perc.append(UniflocVBA.calc_ESP_eff_fr(i, num_stages = num_stages, pump_id = pump_id) * 100)
esp_curve = pd.DataFrame({'Напор, м/100': h_m, 'Мощность, кВт': power_wt, 'КПД, %.': efficency_perc})
esp_curve.index = q_m3day



esp_curve.head()

esp_traces = pw.create_traces_list_for_all_columms(esp_curve, chosen_mode='lines+markers', use_gl=False)
pw.plot_func(esp_traces, 'Характеристика ЭЦН', 'Характеристика ЭЦН.html')

In [ ]:
from plotly.offline import download_plotlyjs, plot
import plotly.figure_factory as ff
import plotly.graph_objs as go

In [ ]:
def plot_corr_heatmap(df, filename_str):
    corrs = df.corr()
    figure = ff.create_annotated_heatmap(
       z=corrs.values,
       x=list(corrs.columns),
       y=list(corrs.index),
       annotation_text=corrs.round(2).values,
       showscale=True)
    figure.layout.update(
        margin=go.layout.Margin(
            l=400,
            r=50,
            b=100,
            t=250
        ))
    filename_str +='.html'
    plot(figure,filename=filename_str)
    
    
    
def make_dimensionless_df(df):
    result_df_dimensionless = df.copy()
    for i in result_df_dimensionless.columns:
        if i == 'Время':
            del result_df_dimensionless['Время']
        else:
            new_new = result_df_dimensionless[i]/result_df_dimensionless[i].max()
            result_df_dimensionless[i] = new_new

    return result_df_dimensionless

def plot_scatterplotmatrix(df, file_name):
    figure = ff.create_scatterplotmatrix(df, diag='histogram')
    figure.layout.update(width=2000, height=1500)
    figure.layout.update(font=dict(size=7))
    
    plot(figure,filename=file_name)

In [ ]:
nedeed_param_list = ['Q ж, м3/сут (Модель) (ADAPT)', 'К. калибровки по напору - множитель (Модель) (ADAPT)',
                    'К. калибровки по мощности - множитель (Модель) (ADAPT)', 
                     'ГФ (Модель) (ADAPT)',
                     'Обв, % (Модель) (ADAPT)',
                      'F тока, ГЦ (Модель) (ADAPT)',
                     'Мощность, передаваемая СУ (Модель) (ADAPT)',
                     'КПД ЭЦН, д.ед. (Модель) (ADAPT)',
                     'Перепад давления в ЭЦН, атм (Модель) (ADAPT)',
                     'P буф., атм (Модель) (ADAPT)'
                       ]
small_overall_data = overall_data[nedeed_param_list]

In [ ]:
plot_corr_heatmap(small_overall_data, 'small_overall_data')

In [ ]:
dimenless_small_overall_data = make_dimensionless_df(small_overall_data)

In [ ]:
dimenless_small_overall_data.head()

In [ ]:
plot_scatterplotmatrix(small_overall_data, 'small_overall_data_scatterplotmatrix')